# MLlib_RDD_Based

In [1]:
import findspark
findspark.init()

## Thêm các thư viện cần thiết

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.mllib.classification import LabeledPoint, LogisticRegressionWithLBFGS
from pyspark.mllib.linalg import Vectors
import os
import sys

os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

## Tạo session

In [3]:
spark=SparkSession.builder.master("local")\
    .appName("mllib_rdd_based")\
    .config("spark.some.config.option", "some-value")\
    .getOrCreate()

## Đọc dữ liệu

In [4]:
lines=spark.sparkContext.textFile("creditcard.csv")
lines.count()

284808

In [5]:
header=lines.first()

In [6]:
raw_data=lines.filter(lambda line: line !=header)
data=raw_data.map(lambda lines: lines.split(","))\
            .map(lambda line: tuple([float(x) for x in line[0:-1]]+[float(line[-1].strip('"'))]))

In [7]:
for row in data.take(5):
    print(row)

(0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62, 0.0)
(0.0, 1.19185711131486, 0.26615071205963, 0.16648011335321, 0.448154078460911, 0.0600176492822243, -0.0823608088155687, -0.0788029833323113, 0.0851016549148104, -0.255425128109186, -0.166974414004614, 1.61272666105479, 1.06523531137287, 0.48909501589608, -0.143772296441519, 0.635558093258208, 0.463917041022171, -0.114804663102346, -0.183361270123994, -0.145783041325259, -0.0690831352230203, -0.225775248033138, -0.638671952771851, 0.

## Tiền xử lý dữ liệu

### Loại bỏ dữ liệu lặp

In [8]:
data.count()  

284807

In [9]:
data=data.distinct()
data.count()

283726

### Loại bỏ dữ liệu thiếu

In [10]:
data = data.filter(lambda line: all(x is not None and x != '' for x in line))

In [11]:
data.count()

283726

## Chuẩn hóa dữ liệu Min-Max

In [12]:
features = data.map(lambda line: line[0:-1])

min_features=features.reduce(lambda x, y: [min(x[i], y[i]) for i in range(len(x))])
max_features=features.reduce(lambda x, y: [max(x[i], y[i]) for i in range(len(x))])

normalized_data = data.map(lambda line: tuple([(line[i] - min_features[i]) / (max_features[i] - min_features[i]) for i in range(len(line)-1)]+[line[-1]]))

normalized_data.count()

283726

## Tách dữ liệu

In [13]:
train_0,val_0,test_0= normalized_data.filter(lambda x: x[-1] == 0.0).randomSplit([0.7,0.15,0.15], seed=42)
train_1,val_1,test_1= normalized_data.filter(lambda x: x[-1] == 1.0).randomSplit([0.7,0.15,0.15], seed=42)
train=train_0.union(train_1)
val=val_0.union(val_1)
test=test_0.union(test_1)

## Chuyển thành RDD LabeledPoint

In [14]:
train_data=train.map(lambda line: LabeledPoint(line[-1], Vectors.dense(line[0:-1])))
val_data=val.map(lambda line: LabeledPoint(line[-1], Vectors.dense(line[0:-1])))
test_data=test.map(lambda line: LabeledPoint(line[-1], Vectors.dense(line[0:-1])))


## Huấn luyện mô hình

- Sử dụng vòng lặp [100, 200, 500]
- Sử dụng regParam [0.01, 0.1, 0.5]
- Sử dụng thang đo recall trên label 1 để quyết định mô hình tốt nhất

In [15]:
iterations=[20,50,100]
regParams=[0, 0.01, 0.1]

best_recall=0.0
best_model=None
best_params=None
best_metrics=None

In [16]:
from pyspark.mllib.evaluation import MulticlassMetrics
for i in iterations:
    for j in regParams:
        print(f"Training with Iterations: {i}, RegParams {j}")
        model = LogisticRegressionWithLBFGS.train(train_data,iterations=i, regParam=j, intercept=True,numClasses=2)
        
        predictions = val_data.map(lambda p: (float(model.predict(p.features)),p.label))

        metrics = MulticlassMetrics(predictions)

        recall = metrics.recall(1.0)
        
        if recall > best_recall:
            best_recall = recall
            best_model = model
            best_metrics = metrics
            best_params = (i, j)
        print(f"Iterations: {i}, RegParams {j}, Recall: {recall:.4f}")

print(f"Best Recall: {best_recall:.4f} with Iterations: {best_params[0]}, RegParams {best_params[1]}")


Training with Iterations: 20, RegParams 0


C:\Program Files\Apache\Spark\python\pyspark\sql\context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Iterations: 20, RegParams 0, Recall: 0.6125
Training with Iterations: 20, RegParams 0.01
Iterations: 20, RegParams 0.01, Recall: 0.3750
Training with Iterations: 20, RegParams 0.1
Iterations: 20, RegParams 0.1, Recall: 0.2000
Training with Iterations: 50, RegParams 0
Iterations: 50, RegParams 0, Recall: 0.6250
Training with Iterations: 50, RegParams 0.01
Iterations: 50, RegParams 0.01, Recall: 0.3750
Training with Iterations: 50, RegParams 0.1
Iterations: 50, RegParams 0.1, Recall: 0.2000
Training with Iterations: 100, RegParams 0
Iterations: 100, RegParams 0, Recall: 0.6250
Training with Iterations: 100, RegParams 0.01
Iterations: 100, RegParams 0.01, Recall: 0.3750
Training with Iterations: 100, RegParams 0.1
Iterations: 100, RegParams 0.1, Recall: 0.2000
Best Recall: 0.6250 with Iterations: 50, RegParams 0


- Sau khi huấn luyện, mô hình tốt nhất có iteration = 50 và regParam = 0

## Đánh giá mô hình sau khi huấn luyện

In [17]:
print(f"Test Accuracy: {best_metrics.accuracy:.4f}")
print("Test Precision [0, 1]:", [best_metrics.precision(0.0), best_metrics.precision(1.0)])
print("Test Recall [0, 1]:", [best_metrics.recall(0.0), best_metrics.recall(1.0)])
print("Test F1 Score [0, 1]:", [best_metrics.fMeasure(0.0), best_metrics.fMeasure(1.0)])

Test Accuracy: 0.9991
Test Precision [0, 1]: [0.9992982127818846, 0.8620689655172413]
Test Recall [0, 1]: [0.9998127603800965, 0.625]
Test F1 Score [0, 1]: [0.9995554203617474, 0.7246376811594203]


#### Nhận xét

- Độ chính xác tổng thể (Accuracy): 0.9991 - cực kỳ cao.
- Tuy nhiên, đi vào từng lớp:
    - Đối với lớp 0:
        - Đây là lớp chiếm đa số toàn bộ dữ liệu. 
        - Mô hình dự đoán rất tốt với precision, recall và F1 Score gần như tối đa. Kết quả gần như tuyệt đối.
    - Đối với lớp 1:
        - Lớp này chiếm phần nhỏ trong bộ dữ liệu (0.172%).
        - Số liệu đánh giá của mô hình thấp hơn lần lượt là precision 0.862, recall 0.625 và F1 Score 0.725. Đối với bài toán này, việc recall của lớp 1 thấp (chỉ 0.625) là vấn đề nghiêm trọng, bỏ xót nhiều trường hợp gian lận.

    


## Thực hiện dự đoán

In [18]:
predictions= test_data.map(lambda p: (float(best_model.predict(p.features)),p.label))

## Đánh giá mô hình sau khi dự đoán

In [19]:

metrics = MulticlassMetrics(predictions)

accuracy = metrics.accuracy
precision_1 = metrics.precision(1.0)
recall_1 = metrics.recall(1.0)
f1_score_1 = metrics.fMeasure(1.0)

precision_0 = metrics.precision(0.0)
recall_0 = metrics.recall(0.0)
f1_score_0 = metrics.fMeasure(0.0)

print(f"Test Accuracy: {accuracy:.4f}")
print("Test Precision [0, 1]:", [precision_0, precision_1])
print("Test Recall [0, 1]:", [recall_0, recall_1])
print("Test F1 Score [0, 1]:", [f1_score_0, f1_score_1])




Test Accuracy: 0.9994
Test Precision [0, 1]: [0.9995257291913683, 0.8636363636363636]
Test Recall [0, 1]: [0.9998576715058355, 0.6551724137931034]
Test F1 Score [0, 1]: [0.9996916727936817, 0.7450980392156864]


#### Nhận xét

- Tuy kết quả recall của lớp 1 có sự cải thiện song đây vẫn là một kết quả không tốt. Để cải thiện việc này, chúng ta cần oversampling lớp 1 hoặc undersampling lớp 0.

## Lưu kết quả

In [20]:
result=test_data.map(lambda p: (p.features, p.label, float(best_model.predict(p.features)))).toDF(["features", "label", "prediction"])

for row in result.take(5):
    print(row)

Row(features=DenseVector([0.0004, 0.974, 0.7635, 0.8613, 0.3362, 0.7586, 0.2661, 0.2625, 0.7881, 0.506, 0.4997, 0.2448, 0.7419, 0.3978, 0.6247, 0.209, 0.4086, 0.7598, 0.5516, 0.5599, 0.5789, 0.5559, 0.4785, 0.6669, 0.4625, 0.596, 0.336, 0.418, 0.314, 0.0027]), label=0.0, prediction=0.0)
Row(features=DenseVector([0.0005, 0.9542, 0.7726, 0.8712, 0.2865, 0.7632, 0.2655, 0.2649, 0.7878, 0.47, 0.501, 0.2427, 0.7024, 0.4225, 0.6378, 0.3833, 0.4245, 0.7467, 0.6338, 0.6166, 0.5803, 0.565, 0.5522, 0.6636, 0.4013, 0.5434, 0.5195, 0.4201, 0.3165, 0.0001]), label=0.0, prediction=0.0)
Row(features=DenseVector([0.0006, 0.9473, 0.7779, 0.8654, 0.2475, 0.7649, 0.2563, 0.2694, 0.7855, 0.4409, 0.5077, 0.3763, 0.7353, 0.4598, 0.655, 0.3453, 0.4595, 0.7139, 0.6563, 0.5814, 0.5813, 0.5591, 0.4904, 0.6663, 0.4524, 0.5645, 0.4304, 0.4191, 0.3152, 0.0006]), label=0.0, prediction=0.0)
Row(features=DenseVector([0.0009, 0.9319, 0.7561, 0.8506, 0.3111, 0.7483, 0.2825, 0.2754, 0.7923, 0.4404, 0.4902, 0.3441, 0.737

In [21]:
result.coalesce(1).write.mode("overwrite").parquet("results")